In [1]:
import dev.langchain4j.agent.tool.P
import dev.langchain4j.agent.tool.Tool
import dev.langchain4j.agent.tool.ToolExecutionRequest
import dev.langchain4j.data.message.SystemMessage
import dev.langchain4j.data.message.ToolExecutionResultMessage
import dev.langchain4j.data.message.UserMessage
import kotlinx.serialization.json.Json
import kotlinx.serialization.json.jsonObject
import kotlinx.serialization.json.jsonPrimitive

data class Contact(val id: Int, val name: String, val phoneNumber: String)

data class Money(val sum: Int, val currency: String)

object BankingAppTools {
    val contactList = listOf(
        Contact(100, "Alice", "+49 151 23456789"),
        Contact(101, "Bob", "+49 89 98765432"),
        Contact(102, "Charlie", "+36 20 123 4567"),
        Contact(103, "Daniel", "+44 7911 123456"),
    )
    val contactMap = contactList.associateBy { it.id }

    private const val DEFAULT_CURRENCY = "EUR"

    @Tool("Send the specified sum (the amount in default user currency) to a recipient contact by a given id. Verify that the recipient is a valid contact and the user has sufficient funds. Convert the sum to the default currency of the recipient before sending.")
    fun sendMoney(
        @P("the ID of the current user") userID: Int,
        @P("the sum of money to be sent in the default currency") sum: Int,
        @P("the ID of the recipient contact") recipientID: Int,
        @P("the purpose of the transaction") purpose: String
    ): String {
        val user = "${contactMap[recipientID]?.name} (${contactMap[recipientID]?.phoneNumber})"
        println("Please confirm that you are sending $sum $DEFAULT_CURRENCY to $user with the purpose '$purpose'.")
        return "Money was sent."
    }

    @Tool("Get the list of contacts of a current user.")
    fun getContactList(userID: Int): List<Contact> {
        return contactList
    }

    @Tool("Get the balance of a current user.")
    fun getBalance(userID: Int): Money = Money(10, DEFAULT_CURRENCY)

    @Tool("Get the information about the exchange rate between two currencies.")
    fun getExchangeRate(
        @P("The 3-letter ISO currency code representing the base currency you want to convert from (e.g., \"USD\" for US Dollar, \"EUR\" for Euro).") from: String,
        @P("The 3-letter ISO currency code representing the target currency you want to convert to (e.g., \"GBP\" for British Pound, \"JPY\" for Japanese Yen).") to: String,
    ): String = when (from to to) {
        "EUR" to "USD" -> "1.1"
        "EUR" to "GBP" -> "0.86"
        "GBP" to "EUR" -> "1.16"
        "USD" to "EUR" -> "0.9"
        else -> "No information about exchange rate available."
    }
}

fun convertJsonToMap(jsonString: String): Map<String, Any> =
    kotlinx.serialization.json.Json.parseToJsonElement(jsonString)
        .jsonObject
        .mapValues { it.value.jsonPrimitive.content }

interface BankingAssistant {
    @dev.langchain4j.service.SystemMessage(
        """
     You are an AI assistant designed to help users efficiently and accurately. Your primary goal is to provide helpful, precise, and clear responses.

     You should think step by step in order to fulfill the objective with a reasoning divided in Thought/Action/Observation that can repeat multiple times if needed.

     You should first reflect with ‘Thought: {your_thoughts}’ on the current situation, then (if necessary), call tools. When you accomplished the user task, print 'Done.'

     You are a banking assistant. You're having a conversation with a user with userID=123.
     Check that you can accomplish what the user asks for with the provided tools.
     If not, respond "Can't perform the task".
    """
    )
    fun performTask(task: String): Result<String>
}

val bankingAssistant = AiServices.builder(BankingAssistant::class.java)
    .chatLanguageModel(chat)
    .tools(BankingAppTools)
    .build()

val result = bankingAssistant.performTask("Send 25 pounds to userID=543 for the dinner at the restaurant")
println(result.content())
println(result.toolExecutions())


org.jetbrains.kotlinx.jupyter.exceptions.ReplCompilerException: at Cell In[1], line 28, column 42: Parameter 'userID' is never used
at Cell In[1], line 39, column 24: Parameter 'userID' is never used
at Cell In[1], line 44, column 20: Parameter 'userID' is never used
at Cell In[1], line 81, column 24: Unresolved reference: AiServices
at Cell In[1], line 82, column 24: Unresolved reference: chat